In [ ]:
import requests
from selectolax.parser import HTMLParser
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("mode.chained_assignment", None)
import warnings
warnings.filterwarnings('ignore')

## 正文開始
1. 確認總共有幾個頁面來設定需要抓多少
2. 確認title，價格，店家來源
3. 寫入pandas

In [ ]:
class Feebee:
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
    init_url = "https://feebee.com.tw/all/?inventoryOn=0&ptab=1&sort=d&mode=l&best=&cat%5B%5D=3086&pl=&ph=&q=%E8%BC%AA%E8%83%8E&pr%5B%5D=24hpchome&pr%5B%5D=momoshop&pr%5B%5D=etmall&pr%5B%5D=umall&pr%5B%5D=car1&pr%5B%5D=secar"
    
    def __init__(self):
        self.page_num = self.get_pageNum()
        
    def get_response(self, url):
        html = requests.get(url, headers=headers).text
        return html
        
    def get_pageNum(self):
        init_html = self.get_response(init_url)
        parser = HTMLParser(init_html)
        item_nums = int(parser.css_first("em#result_total").text().replace(",", ""))
        # print(item_nums)
        page_nums = (item_nums / 37) + 1
        return int(page_nums)
    
    def parse_item(self, html):
        df = pd.DataFrame(columns=["價格來源", "名稱規格", "價格"])
        parser = HTMLParser(html)
        for store, title, price in zip(parser.css("span.shop"), parser.css("h3.large"), parser.css("span.price.ellipsis.xlarge")):
            print(store.text().strip(), title.text(), price.text().replace(",", ""))
            data = pd.Series([store.text().strip(), title.text(), price.text().replace(",", "")], index=["價格來源", "名稱規格", "價格"])
            df = df.append(data, ignore_index=True)
        # print(df)
        return df
    

In [ ]:
df_list = []
crawler = Feebee()
page_nums = crawler.page_num
for page in range(1, page_nums + 1):
    print("第" + str(page) + "頁")
    url = "https://feebee.com.tw/all/%E8%BC%AA%E8%83%8E/?cat[]=3086&pr[]=24hpchome&pr[]=momoshop&pr[]=etmall&pr[]=umall&pr[]=car1&pr[]=secar&ptab=1&page="  + str(page)
    html = crawler.get_response(url)
    data = crawler.parse_item(html)
    df_list.append(data)
result = pd.concat(df_list).reset_index(drop=True).drop_duplicates()

In [ ]:
result = result.drop_duplicates()

In [ ]:
result.to_excel(r"C:\Users\kc.hsu\Desktop\tests.xlsx", index=False)